In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv('Grounds.csv')

In [3]:
data.head()

,Name,Length (meters),Width (meters),Average Wind speed (kmph),Pitch Type,Average Score,Country
0,Melbourne Cricket Ground,173,148,12,Green,222,Australia
1,Sydney Cricket Ground,156,154,15,Green,223,Australia
2,Adelaide Oval,190,126,14,Green,230,Australia
3,The Gabba,170,150,15,Green,228,Australia
4,WACA Ground,149,139,17,Dead,199,Australia


In [4]:
data.columns

Index(['Name', 'Length (meters)', 'Width (meters)',
       'Average Wind speed (kmph)', 'Pitch Type', 'Average Score',
       '    Country'],
      dtype='object')

In [5]:
data.rename(columns = {'    Country':'country'}, inplace = True)

In [6]:
data.columns

Index(['Name', 'Length (meters)', 'Width (meters)',
       'Average Wind speed (kmph)', 'Pitch Type', 'Average Score', 'country'],
      dtype='object')

In [7]:
onehot_columns = ['country','Pitch Type']
onehot_df = data[onehot_columns]
onehot_df = pd.get_dummies(onehot_df, columns = onehot_columns)
score_onehot_drop = data.drop(onehot_columns, axis = 1)
score_onehot = pd.concat([score_onehot_drop, onehot_df], axis = 1)

In [8]:
score_onehot.columns

Index(['Name', 'Length (meters)', 'Width (meters)',
       'Average Wind speed (kmph)', 'Average Score', 'country_       India',
       'country_      Dubai', 'country_      England', 'country_     Pakistan',
       'country_    Australia', 'country_    Sri Lanka',
       'country_  Afghanistan', 'country_  Bangladesh',
       'country_  New Zealand', 'country_  South Africa',
       'country_  West Indies', 'Pitch Type_Dead', 'Pitch Type_Dusty',
       'Pitch Type_Green'],
      dtype='object')

In [9]:

score_onehot.head()

,Name,Length (meters),Width (meters),Average Wind speed (kmph),Average Score,country_ India,country_ Dubai,country_ England,country_ Pakistan,country_ Australia,country_ Sri Lanka,country_ Afghanistan,country_ Bangladesh,country_ New Zealand,country_ South Africa,country_ West Indies,Pitch Type_Dead,Pitch Type_Dusty,Pitch Type_Green
0,Melbourne Cricket Ground,173,148,12,222,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,Sydney Cricket Ground,156,154,15,223,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,Adelaide Oval,190,126,14,230,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,The Gabba,170,150,15,228,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,WACA Ground,149,139,17,199,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [10]:
score_onehot=score_onehot.drop('Name',1)

In [11]:
score_onehot.head()

,Length (meters),Width (meters),Average Wind speed (kmph),Average Score,country_ India,country_ Dubai,country_ England,country_ Pakistan,country_ Australia,country_ Sri Lanka,country_ Afghanistan,country_ Bangladesh,country_ New Zealand,country_ South Africa,country_ West Indies,Pitch Type_Dead,Pitch Type_Dusty,Pitch Type_Green
0,173,148,12,222,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,156,154,15,223,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,190,126,14,230,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,170,150,15,228,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,149,139,17,199,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [13]:
X = score_onehot.drop("Average Score",1)   
y = score_onehot["Average Score"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.20, random_state=42)

In [15]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [16]:
y_pred = regressor.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)


RMSE: 39.47673593396495


In [17]:
def testing_rf(data):
    
    data_list = list((np.array(data)).reshape(-1))

    df = pd.DataFrame()
    col_names = ['f%d' % i for i in range(17)]
    df = df.append(pd.Series(data=data_list, index=col_names), ignore_index=True)

    result1 = regressor.predict(df)
    
    return np.around(int(result1))

In [18]:
from numpy import asarray

pred = testing_rf([120,112,20,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
pred

298

In [19]:
import pickle

pickle_out = open("rfClassifier.pkl","wb")
pickle.dump(regressor,pickle_out)
pickle_out.close()

In [20]:
pickle_in=open('rfClassifier.pkl','rb')
classifier = pickle.load(pickle_in)

In [21]:
def saved_testing_rf(data):
    
    data_list = list((np.array(data)).reshape(-1))

    df = pd.DataFrame()
    col_names = ['f%d' % i for i in range(17)]
    df = df.append(pd.Series(data=data_list, index=col_names), ignore_index=True)

    result1 = classifier.predict(df)
    
    return np.around(int(result1))

In [22]:
from numpy import asarray

pred = saved_testing_rf([120,112,20,0,0,0,0,0,1,0,0,0,0,0,1,0,0])
pred

298

In [ ]:
length= 120
width=112
avgWind=20
country= [0,0,0,0,0,1,0,0,0,0,0]
pitch=[1,0,0]

dataz=[]

dataz.extend([length])
dataz.extend([width])
dataz.extend([avgWind])
dataz=dataz+country
dataz=dataz+pitch

print(dataz)